## Connecting libraries ##

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from data_manager import DataManager
from rating_manager import RatingManager
from manifold_manager import ManifoldManager

## Functions ##

### Functions for rendering ###

In [ ]:

def plot_ellipsoid(ax, mean, cov, color, n_std=2):
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=color, edgecolor='black', alpha=0.2)
    
    scale_x = np.sqrt(cov[0, 0]) * n_std
    scale_y = np.sqrt(cov[1, 1]) * n_std
    transf = transforms.Affine2D().rotate_deg(45).scale(scale_x, scale_y).translate(mean[0], mean[1])
    ellipse.set_transform(transf + ax.transData)
    ax.add_patch(ellipse)

def plot_condition_data(data, ellipsoid_error=False,
                        ellipsoid_cls='run',
                        equal_limits = None,
                        render_manifold=True,
                        render_only_first_projection=False):
    colors = plt.cm.viridis(np.linspace(0, 1, 3))
    classes = ['sit', 'groom', 'run']

    if render_manifold:

        fig = plt.figure(figsize=(7, 5))
        ax = fig.add_subplot(111, projection='3d')

        for cls, color in zip(classes, colors):
            subset = data[data['condition'] == cls]
            ax.scatter(subset['C1'], subset['C2'], subset['C3'], color=color, label=f'Class {cls}')

        ax.set_xlabel('С1')
        ax.set_ylabel('С2')
        ax.set_zlabel('С3')
        ax.legend()
        plt.savefig('man.png')
        plt.show()


    if render_only_first_projection:
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        axs = [ax]
    else:
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    for cls, color in zip(classes, colors):
        subset = data[data['condition'] == cls]
        axs[0].scatter(subset['C1'], subset['C2'], color=color, label=f'Class {cls}')
        
        if ellipsoid_error and len(subset) > 1 and cls == ellipsoid_cls:
            cov = np.cov(subset[['C1', 'C2']].T)
            mean = subset[['C1', 'C2']].mean().values
            plot_ellipsoid(axs[0], mean, cov, color)

    axs[0].set_xlabel('C1')
    axs[0].set_ylabel('C2')
    axs[0].legend()

    if render_only_first_projection:
        axs[0].set_xlim(*equal_limits)
        axs[0].set_ylim(*equal_limits)
        return

    for cls, color in zip(classes, colors):
        subset = data[data['condition'] == cls]
        axs[1].scatter(subset['C1'], subset['C3'], color=color, label=f'Class {cls}')
        
        if ellipsoid_error and len(subset) > 1 and cls == ellipsoid_cls:
            cov = np.cov(subset[['C1', 'C3']].T)
            mean = subset[['C1', 'C3']].mean().values
            plot_ellipsoid(axs[1], mean, cov, color)

    axs[1].set_xlabel('C1')
    axs[1].set_ylabel('C3')
    axs[1].legend()

    for cls, color in zip(classes, colors):
        subset = data[data['condition'] == cls]
        axs[2].scatter(subset['C2'], subset['C3'], color=color, label=f'Class {cls}')
        
        if ellipsoid_error and len(subset) > 1 and cls == ellipsoid_cls:
            cov = np.cov(subset[['C2', 'C3']].T)
            mean = subset[['C2', 'C3']].mean().values
            plot_ellipsoid(axs[2], mean, cov, color)

    axs[2].set_xlabel('C2')
    axs[2].set_ylabel('C3')
    axs[2].legend()

    if equal_limits:
        for ax in axs:
            ax.set_xlim(*equal_limits)
            ax.set_ylim(*equal_limits)

    plt.show()


In [ ]:
def plot_polar_data(data):

    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(data['C1'], data['C2'], data['C3'], c=data['theta'], cmap='viridis')
    ax.set_xlabel('C1')
    ax.set_ylabel('C2')
    ax.set_zlabel('C3')
    ax.set_title('3D Plot')
    fig.colorbar(sc, ax=ax, label='Parameter')


    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    sc_xy = axs[0].scatter(data['C1'], data['C2'], c=data['theta'], cmap='viridis')
    axs[0].set_xlabel('C1')
    axs[0].set_ylabel('C2')
    axs[0].set_title('C1 C2 Projection')
    fig.colorbar(sc_xy, ax=axs[0], label='Parameter')


    sc_xz = axs[1].scatter(data['C1'], data['C3'], c=data['theta'], cmap='viridis')
    axs[1].set_xlabel('C1')
    axs[1].set_ylabel('C3')
    axs[1].set_title('C1 C3 Projection')
    fig.colorbar(sc_xz, ax=axs[1], label='Parameter')


    sc_yz = axs[2].scatter(data['C2'], data['C3'], c=data['theta'], cmap='viridis')
    axs[2].set_xlabel('C2')
    axs[2].set_ylabel('C3')
    axs[2].set_title('C2 C3 Projection')
    fig.colorbar(sc_yz, ax=axs[2], label='Parameter')

    plt.tight_layout()
    plt.show()

### Functions for calculating ellipse parameters ###

In [ ]:
def calculate_square_from_data(data):
    cov_matrix = np.cov(data, rowvar=False)

    eigenvalues, _ = np.linalg.eigh(cov_matrix)

    a = np.sqrt(max(eigenvalues))
    b = np.sqrt(min(eigenvalues))

    return a * b


def calculate_eccentricity_from_data(data):
    cov_matrix = np.cov(data, rowvar=False)
    eigenvalues, _ = np.linalg.eigh(cov_matrix)

    a = np.sqrt(max(eigenvalues))
    b = np.sqrt(min(eigenvalues))

    return (1 - (b ** 2 / a ** 2))**0.5

def calculate_eccentricity_for_manifold(manifold):
    return (
        calculate_eccentricity_from_data(manifold[['C1', 'C2']]),
        calculate_eccentricity_from_data(manifold[['C1', 'C3']]),
        calculate_eccentricity_from_data(manifold[['C2', 'C3']])
    )

def calculate_square_for_manifold(manifold):
    return (
        calculate_square_from_data(manifold[['C1', 'C2']]),
        calculate_square_from_data(manifold[['C1', 'C3']]),
        calculate_square_from_data(manifold[['C2', 'C3']])
    )

### Functions for compressing state data series ###

In [ ]:
def get_condition_sample_data(final_length, tracking): 
    missing_frames = 9000 - tracking.shape[0]
    condition = pd.concat([
        pd.Series(['' for i in range(missing_frames // 2)]),
        tracking.loc[:, ['groom', 'run', 'sit', 'stand']].idxmax(axis=1).fillna(''),
        pd.Series(['' for i in range(missing_frames // 2 + 1)])
        ], ignore_index=True)
    condition = pd.Series([condition[i*(9000 // final_length):(i+1)* (9000 // final_length)].mode()[0] for i in range(final_length)])
    return condition.rename('condition')

In [ ]:
def get_polar_sample_data(final_length, tracking): 
    missing_frames = 9000 - tracking.shape[0]
    
    r = pd.concat([
        pd.Series([0 for i in range(missing_frames // 2)]),
        pd.Series(np.sqrt(tracking['X, px']**2 + tracking['Y, px']**2)),
        pd.Series([0 for i in range(missing_frames // 2 + 1)])
        ], ignore_index=True)
    
    theta = pd.concat([
        pd.Series([0 for i in range(missing_frames // 2)]),
        pd.Series(np.degrees(np.arctan2(tracking['Y, px'], tracking['X, px']))),
        pd.Series([0 for i in range(missing_frames // 2 + 1)])
        ], ignore_index=True)

    r = pd.Series([r[i*(9000 // final_length):(i+1)* (9000 // final_length)].mean() for i in range(final_length)])
    theta = pd.Series([theta[i*(9000 // final_length):(i+1)* (9000 // final_length)].mean() for i in range(final_length)])
    return pd.concat([r, theta], axis=1).rename(columns={0: 'r', 1: 'theta'})

## Connecting data ##

In [ ]:
WT_mice = [3, 5, 7, 106, 102, 109, 110]
transgenic_mice_drug = [12, 13, 111, 115, 113, 114]
transgenic_mice_without_drug = [1, 15, 16, 117, 123, 118]

In [ ]:
WT_mice_dict = {3: [1, 2, 3, 4], 5: [1, 3, 4], 7:[1, 2, 3, 4], 102: [1, 2], 106: [1, 2], 109: [1, 2], 110: [2]}
transgenic_mice_drug_dict = {12: [1, 2, 3, 4], 13: [1, 2, 3, 4], 111: [1, 2, 4], 113: [1, 2], 114:[2], 115: [1, 2, 4]}
transgenic_mice_without_drug_dict = {1: [1, 2, 3, 4], 15: [1, 2, 3, 4], 16: [1, 2, 4], 117: [1, 2, 4], 118: [1, 2], 123: [1, 2]}

mice = {}
mice.update(WT_mice_dict)
mice.update(transgenic_mice_drug_dict)
mice.update(transgenic_mice_without_drug_dict)

In [ ]:
data_manager = DataManager()
for mouse, sessions in mice.items():
    for session in sessions:
        data_manager.load_data(f'data/alzheimer data/mouse {mouse}/session {session}/minian/C.zarr', mouse)
    time_sample_size = 30

In [ ]:
data_map = data_manager.get_time_split_data(time_sample_size, average_samples=False)

In [ ]:
mice_tracking = {}
for mouse, sessions in mice.items():
    for session in sessions:
        if mouse not in mice_tracking:
            mice_tracking[mouse] = [pd.read_excel(f'data/mouse tracking/mouse{mouse}/session_{session}.xlsx')]
        else:
            mice_tracking[mouse].append(pd.read_excel(f'data/mouse tracking/mouse{mouse}/session_{session}.xlsx'))

## Building manifolds and connecting to mouse behavior ##

### Build once ###

In [ ]:
manifold_manager = ManifoldManager()
manifold_map = {}
for tag, sessions in data_map.items():
    for session in sessions:
        if tag not in manifold_map:
            manifold_map[tag] = [manifold_manager.t_SNE(session)]
        else:
            manifold_map[tag].append(manifold_manager.t_SNE(session))

In [ ]:
data = {}
for mouse, sessions in mice_tracking.items():
    for i, session in enumerate(sessions):
        mouse_tracking = pd.concat([pd.DataFrame(manifold_map[mouse][i], columns=['C1', 'C2', 'C3']),
            get_condition_sample_data(manifold_map[mouse][i].shape[0], mice_tracking[mouse][i]),
            get_polar_sample_data(manifold_map[mouse][i].shape[0], mice_tracking[mouse][i])
            ], axis=1)
        if mouse not in data:
            data[mouse] = [mouse_tracking]
        else:
            data[mouse].append(mouse_tracking)
            

### Building manifolds several times, followed by averaging ###

In [ ]:
from tqdm import tqdm

rating_manager = RatingManager()

square_data = {}
eccentricities_data = {}
intraclaster_data = {}

for i in tqdm(range(1), desc='Processing'):
    data_manager = DataManager()
    for mouse, sessions in mice.items():
        for session in sessions:
            data_manager.load_data(f'data/alzheimer data/mouse {mouse}/session {session}/minian/C.zarr', mouse)

    data_map = data_manager.get_time_split_data(time_sample_size, average_samples=False)
    
    time_sample_size = 30
    manifold_manager = ManifoldManager()
    manifold_map = {}
    for tag, sessions in data_map.items():
        for session in sessions:
            if tag not in manifold_map:
                manifold_map[tag] = [manifold_manager.t_SNE(session)]

            else:
                manifold_map[tag].append(manifold_manager.t_SNE(session))


    data = {}
    for mouse, sessions in mice_tracking.items():
        for i, session in enumerate(sessions):
            mouse_tracking = pd.concat([pd.DataFrame(manifold_map[mouse][i], columns=['C1', 'C2', 'C3']),
                get_condition_sample_data(manifold_map[mouse][i].shape[0], mice_tracking[mouse][i]),
                get_polar_sample_data(manifold_map[mouse][i].shape[0], mice_tracking[mouse][i])
                ], axis=1)
            if mouse not in data:
                data[mouse] = [mouse_tracking]
            else:
                data[mouse].append(mouse_tracking)

    for mouse, sessions in mice.items():
        for i, session in enumerate(sessions):
            condition = 'sit'

            if data[mouse][i][data[mouse][i]['condition'] == condition].empty or \
            data[mouse][i][data[mouse][i]['condition'] == condition].shape[0] <= 1:
                eccentricities = (0, 0, 0)
                squares = (0, 0, 0)
            else:
                eccentricities = calculate_eccentricity_for_manifold(data[mouse][i][data[mouse][i]['condition'] == condition])
                squares = calculate_square_for_manifold(data[mouse][i][data[mouse][i]['condition'] == condition])

            intraclaster = rating_manager.intracluster_distance(data[mouse][i][['C1', 'C2', 'C3']])

            
            if mouse not in square_data:
                square_data[mouse] = {session : [squares]}
            elif session not in square_data[mouse]:
                square_data[mouse][session] = [squares]
            else:
                square_data[mouse][session].append(squares)

            if mouse not in eccentricities_data:
                eccentricities_data[mouse] = {session : [eccentricities]}
            elif session not in eccentricities_data[mouse]:
                eccentricities_data[mouse][session] = [eccentricities]
            else:
                eccentricities_data[mouse][session].append(eccentricities)

            if mouse not in intraclaster_data:
                intraclaster_data[mouse] = {session : [intraclaster]}
            elif session not in intraclaster_data[mouse]:
                intraclaster_data[mouse][session] = [intraclaster]
            else:
                intraclaster_data[mouse][session].append(intraclaster)

 ### Getting a dataframe with values of eccentricities and areas for all mice for all iterations ###

In [ ]:
def get_param_df(param_data):
    param_df = []
    for mouse_type, mice_num in [('wild', WT_mice), ('transgenic', transgenic_mice_without_drug), 
                                ('treated', transgenic_mice_drug)]:
        for mouse_num in mice_num:
            for session, param_session in param_data[mouse_num].items():
                param_df.append(list(np.array(param_session).mean(axis=1)) + [mouse_type, mouse_num, session])

    return pd.DataFrame(param_df).rename({10: 'type', 11: 'mouse_num', 12: 'session_num'}, axis=1)

In [ ]:
square_df = get_param_df(square_data)

In [ ]:
square_df.shape

In [ ]:
square_df.to_excel(f'square_{condition}_exp_10.xlsx')

In [ ]:
eccentricities_df = get_param_df(eccentricities_data)

In [ ]:
eccentricities_df.to_excel(f'eccentricities_{condition}_exp_10.xlsx')

## Building a dataframe with session attributes without averaging over iterations ##

In [ ]:
feature_list_data = []
for mouse, sessions in mice.items():
    for session in sessions:
        feature_list_data.append([mouse, session] +
                                 intraclaster_data[mouse][session] +
                                 list(np.mean(np.array(square_data[mouse][session]), axis=1)) +
                                 list(np.mean(np.array(eccentricities_data[mouse][session]), axis=1))
                                )
                

In [ ]:
columns_name_dict = {0: 'mouse', 1: 'session'}
columns_name_dict.update({i: f'intraclaster_distance_{i - 2}' for i in range(2, 12)})
columns_name_dict.update({i: f'square_{i - 12}' for i in range(12, 22)})
columns_name_dict.update({i: f'eccentricity_{i - 22}' for i in range(22, 32)})

feature_data = pd.DataFrame(feature_list_data).rename(columns_name_dict, axis=1)

In [ ]:
def determine_mouse_type(mouse):
    if mouse in transgenic_mice_without_drug:
        return 'transgenic_without_drug'
    elif mouse in transgenic_mice_drug:
        return 'transgenic_with_drug'
    elif mouse in WT_mice:
        return 'wild'

In [ ]:
feature_data['type'] = feature_data['mouse'].apply(determine_mouse_type)

## Building a dataframe with session attributes by averaging all the results ##

In [ ]:
feature_list_data = []
for mouse, sessions in mice.items():
    for session in sessions:
        feature_list_data.append([mouse, session] +
                                 list(np.mean(np.array(square_data[mouse][session]), axis=0)) +
                                 list(np.mean(np.array(eccentricities_data[mouse][session]), axis=0)) +
                                 [np.mean(np.array(intraclaster_data[mouse][session]))]
                                )
                

In [ ]:
feature_data = pd.DataFrame(feature_list_data, columns=['mouse', 'session',
                                                        'square_1', 'square_2',
                                                        'square_3', 'eccentricities_1',
                                                        'eccentricities_2', 'eccentricities_3',
                                                        'intraclaster_distance'])

In [ ]:
def determine_mouse_type(mouse):
    if mouse in transgenic_mice_without_drug:
        return 'transgenic_without_drug'
    elif mouse in transgenic_mice_drug:
        return 'transgenic_with_drug'
    elif mouse in WT_mice:
        return 'wild'

In [ ]:
feature_data['type'] = feature_data['mouse'].apply(determine_mouse_type)

In [ ]:
feature_data['eccentricities_mean'] = feature_data[['eccentricities_1', 'eccentricities_2', 'eccentricities_3']].mean(axis=1)
feature_data['square_mean'] = feature_data[['square_1','square_2','square_3']].mean(axis=1)


## Classification ##

In [ ]:
feature_data_without_drug = feature_data[feature_data['type'] != 'transgenic_with_drug']
feature_data_without_drug['type'] = feature_data_without_drug['type'].apply(lambda x: 0 if x == 'wild' else 1)

In [ ]:
feature_data_without_transgenic = feature_data[feature_data['type'] != 'transgenic_without_drug']
feature_data_without_transgenic['type'] = feature_data_without_transgenic['type'].apply(lambda x: 0 if x == 'wild' else 1)

In [ ]:
feature_data_without_wild = feature_data[feature_data['type'] != 'wild']
feature_data_without_wild['type'] = feature_data_without_wild['type'].apply(lambda x: 1 if x == 'transgenic_without_drug' else 0)

In [ ]:
def train_classifier(feature_type_data, iteration=100):
    result_df = []
    for exp_num in range(1):
        accuracy_score_list = []
        precision_score_list = []
        recall_score_list = []
        f1_score_list = []


        for i in range(iteration):
            X_train, X_test, y_train, y_test = train_test_split(feature_type_data[['intraclaster_distance']],
                                                        feature_type_data['type'],
                                                        test_size=0.3,
                                                        stratify=feature_type_data['type'])
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            model = LogisticRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            accuracy_score_list.append(accuracy_score(y_test, y_pred))
            precision_score_list.append(precision_score(y_test, y_pred))
            recall_score_list.append(recall_score(y_test, y_pred))
            f1_score_list.append(f1_score(y_test, y_pred))
        
        result_df.append([exp_num + 1] + [np.mean(accuracy_score_list),
                                                        np.mean(precision_score_list),
                                                        np.mean(recall_score_list),
                                                        np.mean(f1_score_list)
                                                        ])
    return pd.DataFrame(result_df).rename({0: 'exp_num', 1: 'accuracy', 2: 'precision',
                                                3: 'recall', 4: 'f1'}, axis=1)

**Разделение на трансгеных и пролеченных мышей:**

In [ ]:
feature_data_without_wild.dropna(inplace=True)

In [ ]:
trans_vs_treated_df = train_classifier(feature_data_without_wild)

In [ ]:
trans_vs_treated_df.mean()

**Разделение на трансгеных и диких мышей:**

In [ ]:
feature_data_without_drug.dropna(inplace=True)

In [ ]:
trans_vs_wild_df = train_classifier(feature_data_without_drug)

In [ ]:
trans_vs_wild_df.mean()

**Разделение на диких и пролеченных мышей:**

In [ ]:
feature_data_without_transgenic.dropna(inplace=True)

In [ ]:
wild_vs_treated_df = train_classifier(feature_data_without_transgenic)

In [ ]:
wild_vs_treated_df.mean()

## Building boxplots ##

### Building boxplots by session ###

In [ ]:
healthy_boxplot_data = []
transgenic_boxplot_data = []
drug_boxplot_data = []
for mouse, sessions in square_data.items():
    for session_num, session in sessions.items():
        if mouse in transgenic_mice_without_drug:
            transgenic_boxplot_data.append(np.mean((np.mean(session, axis=0))))

for mouse, sessions in square_data.items():
    for session_num, session in sessions.items():
        if mouse in [12, 13, 111, 115]:
            drug_boxplot_data.append(np.mean((np.mean(session, axis=0))))

for mouse, sessions in square_data.items():
    for session_num, session in sessions.items():
        if mouse in WT_mice:
            healthy_boxplot_data.append(np.mean((np.mean(session, axis=0))))
        

In [ ]:
plt.boxplot([healthy_boxplot_data, transgenic_boxplot_data, drug_boxplot_data],
            labels=['healthy', 'transgenic', 'drug'])

In [ ]:
healthy_boxplot_data = []
transgenic_boxplot_data = []
drug_boxplot_data = []
for mouse, sessions in eccentricities_data.items():
    for session_num, session in sessions.items():
        if mouse in transgenic_mice_without_drug:
            transgenic_boxplot_data.append(np.mean((np.mean(session, axis=0))))

for mouse, sessions in eccentricities_data.items():
    for session_num, session in sessions.items():
        if mouse in [12, 13, 111, 115]:
            drug_boxplot_data.append(np.mean((np.mean(session, axis=0))))

for mouse, sessions in eccentricities_data.items():
    for session_num, session in sessions.items():
        if mouse in WT_mice:
            healthy_boxplot_data.append(np.mean((np.mean(session, axis=0))))
        

In [ ]:
plt.boxplot([healthy_boxplot_data, transgenic_boxplot_data, drug_boxplot_data],
            labels=['healthy', 'transgenic', 'drug'])

### Building boxplots based on mice ###

In [ ]:
healthy_boxplot_data = []
transgenic_boxplot_data = []
drug_boxplot_data = []

for mouse, sessions in square_data.items():
    mouse_boxplot_values = []
    if mouse in transgenic_mice_without_drug:
        for session_num, session in sessions.items():
            mouse_boxplot_values.append(np.mean((np.mean(session, axis=0))))
        transgenic_boxplot_data.append(np.mean(mouse_boxplot_values))


for mouse, sessions in square_data.items():
    mouse_boxplot_values = []
    if mouse in transgenic_mice_drug:
        for session_num, session in sessions.items():
            mouse_boxplot_values.append(np.mean((np.mean(session, axis=0))))
        drug_boxplot_data.append(np.mean(mouse_boxplot_values))

for mouse, sessions in square_data.items():
    mouse_boxplot_values = []
    if mouse in WT_mice:
        for session_num, session in sessions.items():
            mouse_boxplot_values.append(np.mean((np.mean(session, axis=0))))
        healthy_boxplot_data.append(np.mean(mouse_boxplot_values))
        

In [ ]:
plt.boxplot([healthy_boxplot_data, transgenic_boxplot_data, drug_boxplot_data],
            labels=['healthy', 'transgenic', 'drug'])

In [ ]:
healthy_boxplot_data = []
transgenic_boxplot_data = []
drug_boxplot_data = []

for mouse, sessions in eccentricities_data.items():
    mouse_boxplot_values = []
    if mouse in transgenic_mice_without_drug:
        for session_num, session in sessions.items():
            mouse_boxplot_values.append(np.mean((np.mean(session, axis=0))))
        transgenic_boxplot_data.append(np.mean(mouse_boxplot_values))


for mouse, sessions in eccentricities_data.items():
    mouse_boxplot_values = []
    if mouse in transgenic_mice_drug:
        for session_num, session in sessions.items():
            mouse_boxplot_values.append(np.mean((np.mean(session, axis=0))))
        drug_boxplot_data.append(np.mean(mouse_boxplot_values))

for mouse, sessions in eccentricities_data.items():
    mouse_boxplot_values = []
    if mouse in WT_mice:
        for session_num, session in sessions.items():
            mouse_boxplot_values.append(np.mean((np.mean(session, axis=0))))
        healthy_boxplot_data.append(np.mean(mouse_boxplot_values))
        

In [ ]:
plt.boxplot([healthy_boxplot_data, transgenic_boxplot_data, drug_boxplot_data],
            labels=['healthy', 'transgenic', 'drug'])

## Plotting graphs with error ellipses ##

In [ ]:
plot_condition_data(pd.concat([data[3][0][['C1', 'C2', 'C3', 'condition']]]),
                    ellipsoid_error=True,
                    equal_limits=[-200, 200],
                    ellipsoid_cls='run',
                    render_manifold=True,
                    render_only_first_projection=True)

In [ ]:
plot_condition_data(data[12][1],
                    ellipsoid_error=True,
                    equal_limits=[-200, 200],
                    ellipsoid_cls='sit',
                    render_manifold=False,
                    render_only_first_projection=True)

In [ ]:
plot_condition_data(data[1][1],
                    ellipsoid_error=True,
                    equal_limits=[-200, 200],
                    ellipsoid_cls='sit',
                    render_manifold=False,
                    render_only_first_projection=True)